# Character-Level LSTM

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Loading Data 

In [2]:
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In [4]:
# int2char maps integers to characters
# char2int maps characters to unique integers
# chars contain all unique character present in text

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([31, 52, 66, 51, 43, 46, 69, 57, 60, 17, 17, 17, 41, 66, 51, 51, 39,
       57, 28, 66, 19, 48, 12, 48, 46,  1, 57, 66, 69, 46, 57, 66, 12, 12,
       57, 66, 12, 48, 34, 46, 20, 57, 46, 11, 46, 69, 39, 57,  2, 13, 52,
       66, 51, 51, 39, 57, 28, 66, 19, 48, 12, 39, 57, 48,  1, 57,  2, 13,
       52, 66, 51, 51, 39, 57, 48, 13, 57, 48, 43,  1, 57, 26, 27, 13, 17,
       27, 66, 39, 59, 17, 17,  3, 11, 46, 69, 39, 43, 52, 48, 13])

## Pre-processing the data

In [6]:
def one_hot_encode(arr, n_labels):
    
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Filling the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Reshaping it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[ 0.  0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.  0.  0.]]]


## Making training mini-batches

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    n_batches = len(arr)//(batch_size * seq_length)
    
    ## Keeping only enough characters to make full batches
    arr = arr[:n_batches * (batch_size * seq_length)]
    
    ## Reshaping into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    ## Iterating over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

---
## Defining the network

In [9]:
# checking if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


### Model structure

In [10]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
        '''
                
        ## outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## dropout layer
        out = self.dropout(r_output)

        # Stacking up LSTM outputs using view
        # contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## fully-connected layer
        out = self.fc(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Training the network

In [11]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encoding the data and making them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            output, h = net(inputs, h)
            
            # loss and backpropagation
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # clip_grad_norm prevent the exploding gradient problem
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train()
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

In [12]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


## Training the model

In [17]:
batch_size = 128
seq_length = 100
n_epochs =  20

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/20... Step: 50... Loss: 3.1440... Val Loss: 3.1161
Epoch: 1/20... Step: 100... Loss: 3.0646... Val Loss: 3.0531
Epoch: 2/20... Step: 150... Loss: 2.5912... Val Loss: 2.5431
Epoch: 2/20... Step: 200... Loss: 2.3842... Val Loss: 2.3571
Epoch: 2/20... Step: 250... Loss: 2.2297... Val Loss: 2.2308
Epoch: 3/20... Step: 300... Loss: 2.1484... Val Loss: 2.1140
Epoch: 3/20... Step: 350... Loss: 2.0716... Val Loss: 2.0269
Epoch: 3/20... Step: 400... Loss: 1.9488... Val Loss: 1.9462
Epoch: 4/20... Step: 450... Loss: 1.8644... Val Loss: 1.8782
Epoch: 4/20... Step: 500... Loss: 1.8640... Val Loss: 1.8245
Epoch: 4/20... Step: 550... Loss: 1.8250... Val Loss: 1.7696
Epoch: 5/20... Step: 600... Loss: 1.7367... Val Loss: 1.7322
Epoch: 5/20... Step: 650... Loss: 1.7080... Val Loss: 1.6892
Epoch: 6/20... Step: 700... Loss: 1.6733... Val Loss: 1.6560
Epoch: 6/20... Step: 750... Loss: 1.6129... Val Loss: 1.6278
Epoch: 6/20... Step: 800... Loss: 1.6254... Val Loss: 1.6051
Epoch: 7/20... Step: 850.

In [18]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        out, h = net(inputs, h)

        # character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [20]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [21]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna shidter
in a crowd and the sort of happiless, and started at the merching the
servorm.

The memories of the sacrace of the meeting had seen some moments, sens and
to be sure that she should go to him to him. The carriage had no means to
him. The more had been delighted to him, so to say.

At the recovinions of his brother was so many. And she went to a
smile, but the same time their wealth and the possible carriage as though
something was standing, when the same ashement of his sort, with his friend,
soonef, and said that he had succeeded him to their sense of socance of
the sort.

"It's true whom I wanted to be in the frait of his brother--and his wife
was so lest found. How are you are free only one of them the carriages and
some ow that you, there'r noting to step the carryange."

"Well it was to bring it?"

"Yes, but that's in your suncertions."

She was not tines of that with their closs, who would have a man
and what they saw that she did not see the proplexed of a long wais

## Loading a checkpoint

In [22]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [23]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
at the presens of with a little carriage; "but will be, and all the
same sort of sinceress is no old standing of it. Why should I do be
nothing on to be districtly one of her, but the linent moment, and
see you are so stalling for you, I shall be nothing?"

She was going about the colonel at the most farsh of a
soft of touch of the same to the same on the part, but at tame in
the more itself telling his sense of without together were such all
though seemed so much there, when he could not see them, and they heard
a married men. Soness turned off to the sound, the conversation took the
chollence of home, but his wife wished a single softer,
he so as he was not talking of herself, and his brother was strange
feeling.

She had at a smile smile.

"The peasants. The country ond host was a strangel of myself immense that he's so
district, thinks of the position."

"You want to say."

"I don't know, I'll go to her in her, and the carriage of insteast in
those teres. There is th